<a href="https://colab.research.google.com/github/kurtjcu/nerf_pl/blob/dev/extract_mesh_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [0]:
!git clone --single-branch --branch https://github.com/kwea123/nerf_pl

%cd /content/nerf_pl
!pip install -r requirements.txt

#%cd /content/nerf_pl/torchsearchsorted
#!pip install .

# Mount your drive (to access data)

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
%cd /content/nerf_pl

import torch
from collections import defaultdict
import numpy as np
import mcubes
import trimesh

from models.rendering import *
from models.nerf import *

from datasets import dataset_dict

from utils import load_ckpt

/content/nerf_pl


# Load model

In [0]:
# Change here #
img_wh = (4032, 3024) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = 'silica' # whatever you want
root_dir = '/content/drive/My Drive/colab/nerf/my/silica' # the folder containing data (images/ and poses_bounds.npy)
ckpt_path = '/content/silica.ckpt' # the model path
###############

kwargs = {'root_dir': root_dir,
          'img_wh': img_wh}
if dataset_name == 'llff':
    kwargs['spheric_poses'] = True
    kwargs['split'] = 'test'
else:
    kwargs['split'] = 'train'
    
chunk = 1024*32
dataset = dataset_dict[dataset_name](**kwargs)

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval();

# Search for tight bounds (trial and error!)

In [0]:
### Tune these parameters until the whole object lies tightly in range with little noise ###
N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1, 1 # left/right range
ymin, ymax = -1, 1 # forward/backward range
zmin, zmax = -2.64, -0.64 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 20. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
############################################################################################

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)
z = np.linspace(zmin, zmax, N)

xyz_ = torch.FloatTensor(np.stack(np.meshgrid(x, y, z), -1).reshape(-1, 3)).cuda()
dir_ = torch.zeros_like(xyz_).cuda()

with torch.no_grad():
    B = xyz_.shape[0]
    out_chunks = []
    for i in range(0, B, chunk):
        xyz_embedded = embedding_xyz(xyz_[i:i+chunk]) # (N, embed_xyz_channels)
        dir_embedded = embedding_dir(dir_[i:i+chunk]) # (N, embed_dir_channels)
        xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
        out_chunks += [nerf_fine(xyzdir_embedded)]
    rgbsigma = torch.cat(out_chunks, 0)
    
sigma = rgbsigma[:, -1].cpu().numpy()
sigma = np.maximum(sigma, 0)
sigma = sigma.reshape(N, N, N)

vertices, triangles = mcubes.marching_cubes(sigma, sigma_threshold)

mesh = trimesh.Trimesh(vertices/N, triangles)
mesh.show()

In [0]:
# You can already export "colorless" mesh if you don't need color
mcubes.export_mesh(vertices, triangles, f"{scene_name}.dae")

# Extract colored mesh
Once you find the best range, now **RESTART** the notebook(Runtime->Restart runtime), and copy the configs to the following cell and execute it.

In [5]:
%cd /content/nerf_pl

# Copy the variables you have above here! ####

img_wh = (4032, 3024) # full resolution of the input images
dataset_name = 'llff' # blender or llff (own data)
scene_name = 'silica' # whatever you want
root_dir = '/content/drive/My Drive/colab/nerf/my/silica' # the folder containing data
ckpt_path = '/content/silica.ckpt' # the model path

N = 128 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
xmin, xmax = -1, 1 # left/right range
ymin, ymax = -1, 1 # forward/backward range
zmin, zmax = -2.64, -0.64 # up/down range
## Attention! the ranges MUST have the same length!
sigma_threshold = 20. # controls the noise (lower=maybe more noise; higher=some mesh might be missing)
###############################################

import os
os.environ['ROOT_DIR'] = root_dir
os.environ['DATASET_NAME'] = dataset_name
os.environ['SCENE_NAME'] = scene_name
os.environ['IMG_SIZE'] = f"{img_wh[0]} {img_wh[1]}"
os.environ['CKPT_PATH'] = ckpt_path
os.environ['N_GRID'] = "512" # final resolution. You can set this number high to preserve more details
os.environ['X_RANGE'] = f"{xmin} {xmax}"
os.environ['Y_RANGE'] = f"{ymin} {ymax}"
os.environ['Z_RANGE'] = f"{zmin} {zmax}"
os.environ['SIGMA_THRESHOLD'] = str(sigma_threshold)
os.environ['OCC_THRESHOLD'] = "0.2" # probably doesn't require tuning. If you find the color is not close
                                    # to real, try to set this number smaller (the effect of this number
                                    # is explained in my youtube video)

!python extract_color_mesh.py \
    --root_dir "$ROOT_DIR" \
    --dataset_name $DATASET_NAME \
    --scene_name $SCENE_NAME \
    --img_wh $IMG_SIZE \
    --ckpt_path $CKPT_PATH \
    --N_grid $N_GRID \
    --x_range $X_RANGE \
    --y_range $Y_RANGE \
    --z_range $Z_RANGE \
    --sigma_threshold $SIGMA_THRESHOLD \
    --occ_threshold $OCC_THRESHOLD

/content/nerf_pl
tcmalloc: large alloc 1073741824 bytes == 0x77da4000 @  0x7fb3e6ccb1e7 0x7fb39b4d65e1 0x7fb39b53b8e0 0x7fb39b530c95 0x7fb39b5c5957 0x50a635 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x59fc4e 0x7fb39b5249ad 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7fb3e68c8b97 0x5b250a
tcmalloc: large alloc 3221225472 bytes == 0x138634000 @  0x7fb3e6ccb1e7 0x7fb39b4d65e1 0x7fb39b539138 0x7fb39b539253 0x7fb39b5d7368 0x7fb39b5d7bc4 0x7fb39b5d7d12 0x566d63 0x59fc4e 0x7fb39b5249ad 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x588d41 0x59fc4e 0x7fb39b5249ad 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef
tcmalloc: large alloc 1610612736 bytes == 0x77da4000 @  0x7fb3e6cadb6b 0x7fb3e6ccd379 0x7fb39bc2b04e 0x7fb39bc2cf4a 0x7fb3d4b1b67b 0x7fb3d476a6be 0x7fb3d49d37b5 0x7fb3d49c57c1 0x7fb3d49c4d0e 0x7fb3d49c57c1 0x7fb3d641a93a 0x7f